In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_user_attributes = _hex_json.loads("{}")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"019c5371-fe99-7003-b788-6a5f3cfa98ce\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Tyler Waugh Payment Funnel Analysis\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

This query pulls all payment status movements for subscription 12622, showing an ideal happy path funnel:

1. PaymentWidgetOpened → user opens the payment portal.
2. PaymentEntered → user enters payment details.
3. PaymentSubmitted → user submits payment.
4. PaymentSuccess → 3rd-party processor approves the payment.
5. Complete → subscription is fully converted to a paid plan.


In [ ]:
# import jinja2
# raw_query = """
#     select
#         *
#     from
#         public.payment_status_log psl
#     join
#         public.payment_status_definitions def
#         on psl.status_id = def.status_id
#     where   
#         subscription_id = '12622'
#     order by
#         subscription_id,
#         movement_date
#     
#     -- Happy Path
# """
# sql_query = jinja2.Template(raw_query).render(vars())

This query pulls all payment status movements for subscription 38844. It shows a failed attempt followed by a successful recovery:

1. PaymentWidgetOpened – user opens the payment portal.
2. PaymentEntered – user enters payment details.
3. PaymentSubmitted – user submits payment / Error on the user side 
4. PaymentEntered – user retries by re‑entering or confirming payment details.
5. PaymentSubmitted – user submits payment again.
6. PaymentSuccess – the second attempt is approved by the payment processor.
7. Complete – subscription is successfully converted to a paid plan.

This subscription ultimately completes successfully, but only after an intermediate error. It’s a good example of a recovered funnel: users hit an error but are able to fix it and continue to completion. You can contrast this with the pure “happy path” subscription (12622) to quantify how often users recover vs. drop off after an error.


In [ ]:
# import jinja2
# raw_query = """
#     select
#         *
#     from
#         public.payment_status_log psl
#     join
#         public.payment_status_definitions def
#         on psl.status_id = def.status_id
#     where   
#         subscription_id = '38844'
#     order by
#         subscription_id,
#         movement_date
# """
# sql_query = jinja2.Template(raw_query).render(vars())

This query pulls all payment status movements for subscription 38499. 
1. PaymentWidgetOpened – user opens the payment portal.
2. PaymentEntered – user enters payment details.
3. PaymentSubmitted – user submits payment.
4. PaymentSuccess – the payment logs an error on the vendor side


The funnel reaches PaymentSuccess, so the core payment itself completes successfully from the user’s perspective.
The Error that occurs afterward is an example of a vendor error — a failure on the 3rd‑party/payment vendor side or in downstream vendor‑driven processes (e.g., settlement, receipt generation, or post‑payment callbacks).There is no recovery recorded after this vendor error (no second submit or success). The flow ends on Error, making this a post‑success vendor error case, not a clean happy path or a recovered funnel. This makes subscription 38499 a useful example when analyzing vendor‑side failures that occur after an apparently successful payment.





In [ ]:
# import jinja2
# raw_query = """
#     select
#         *
#     from
#         public.payment_status_log psl
#     join
#         public.payment_status_definitions def
#         on psl.status_id = def.status_id
#     where   
#         subscription_id = '38499'
#     order by
#         subscription_id,
#         movement_date
# """
# sql_query = jinja2.Template(raw_query).render(vars())

This query rolls up individual payment events into high‑level funnel stages by order year.

Key funnel stages include:
* User Has Not Started Payment Process
* Payment Widget Opened
* Payment Entered
* Payment Submitted
* User Error with Payment Submission
* Payment Sucess With Vendor
* Payment Processing Error With Vendor (vendor‑side failures)
* Complete

How It's Useful 

* Surfaces drop‑off points in the funnel (e.g., how many get stuck at with user or vendor error).
* Separates user‑driven failures from vendor‑driven errors (using current_payment_status alongside max_status).
* Lets you track funnel health over time by comparing distributions across order years (e.g., are vendor errors decreasing year over year?).

In [ ]:
# import jinja2
# raw_query = """
#     with max_status_reached as(
#     select
#         psl.subscription_id,
#         max(psl.status_id) as max_status
#     from 
#         public.payment_status_log psl
#     group by    
#         1
#     )
#     ,
#     payment_funnel_stages as (
#     select
#         subs.subscription_id,
#         date_trunc('year' , order_date) as order_year,
#         current_payment_status,
#         max_status,
#         case when max_status = 1 then'Payment Widget Opened'
#         when max_status = 2 then'Payment Entered'
#         when max_status = 3 and current_payment_status = 0 then'User Error with Payment Submission'
#         when max_status = 3 and current_payment_status != 0 then'Payment Submitted'
#         when max_status = 4 and current_payment_status = 0 then'Payment Processing Error With Vendor'
#         when max_status = 4 and current_payment_status != 0 then'Payment Sucess With Vendor'
#         when max_status = 5 then 'Complete'
#         when max_status is null then 'User Has Not Started Payment Process'
#         end as payment_funnel_stage
#     from
#         public.subscriptions subs
#     left join
#         max_status_reached m
#         on subs.subscription_id = m.subscription_id
#     )
#     select 
#         payment_funnel_stage,
#         order_year, 
#         count(*) as num_subs
#     from
#         payment_funnel_stages
#     group by 
#         1, 2 
#     order by 
#         2 desc 
#     
#     /* We need both max_status field from cte and current_payment_status 
#     in the same query so we can use them together when categorizing subscriptions 
#     
#     left join to not lose the subscriptions that haven't been in the portal yet,
#     as they will not be on max_status_reached
#     */
# """
# sql_query = jinja2.Template(raw_query).render(vars())

This query assigns each subscription to the furthest payment stage it has reached (from “Not Started” through various error states to “Complete”) and counts how many subscriptions fall into each stage by order year. The results highlight that many customers never start the payment process, while most who do eventually complete payment, with relatively few dropping out due to user or vendor errors. However, what this is not showing is that many are are not even in the payment funnel at all and haven't started to pay. 

In [ ]:
# import jinja2
# raw_query = """
#     WITH ordered_status AS (
#         SELECT
#             psl.subscription_id,
#             def.description AS status_description,
#             psl.movement_date,
#             LEAD(def.description) OVER (
#                 PARTITION BY psl.subscription_id
#                 ORDER BY psl.movement_date
#             ) AS next_status_description
#         FROM public.payment_status_log psl
#         JOIN public.payment_status_definitions def
#             ON psl.status_id = def.status_id
#     ),
#     transitions AS (
#         SELECT
#             status_description   AS from_status,
#             next_status_description AS to_status,
#             COUNT(*) AS transition_count
#         FROM ordered_status
#         WHERE next_status_description IS NOT NULL
#         GROUP BY 1, 2
#     )
#     
#     SELECT
#         from_status,
#         to_status,
#         transition_count
#     FROM transitions
#     ORDER BY transition_count DESC;
# """
# sql_query = jinja2.Template(raw_query).render(vars())

This query analyzes where users drop off in the payment funnel and what types of errors occur, broken down by order year.

It classifies each subscription into a high‑level payment funnel stage based on:

* The maximum payment status that subscription has ever reached, and
* The current payment status (to distinguish between user errors vs vendor errors at the same stage).

Then it counts, per year:

* How many subscriptions had user errors submitting payment
* How many had vendor/processing errors
* The total number of errored subscriptions

In [ ]:
# import jinja2
# raw_query = """
#     with max_status_reached as (
#         select
#             psl.subscription_id,
#             max(psl.status_id) as max_status
#         from 
#             public.payment_status_log psl
#         group by    
#             1
#     ),
#     payment_funnel_stages as (
#         select
#             subs.subscription_id,
#             date_trunc('year', order_date) as order_year,
#             current_payment_status,
#             max_status,
#             case 
#                 when max_status = 1 then 'Payment Widget Opened'
#                 when max_status = 2 then 'Payment Entered'
#                 when max_status = 3 and current_payment_status = 0 then 'User Error with Payment Submission'
#                 when max_status = 3 and current_payment_status != 0 then 'Payment Submitted'
#                 when max_status = 4 and current_payment_status = 0 then 'Payment Processing Error With Vendor'
#                 when max_status = 4 and current_payment_status != 0 then 'Payment Sucess With Vendor'
#                 when max_status = 5 then 'Complete'
#                 when max_status is null then 'User Has Not Started Payment Process'
#             end as payment_funnel_stage
#         from
#             public.subscriptions subs
#         left join
#             max_status_reached m
#             on subs.subscription_id = m.subscription_id
#     )
#     select
#         order_year,
#         -- counts of each error type
#         sum(case when payment_funnel_stage = 'User Error with Payment Submission' then 1 else 0 end) as user_errors,
#         sum(case when payment_funnel_stage = 'Payment Processing Error With Vendor' then 1 else 0 end) as vendor_errors,
#         -- optional: total errors
#         sum(case when payment_funnel_stage in (
#                 'User Error with Payment Submission',
#                 'Payment Processing Error With Vendor'
#             ) then 1 else 0 end) as total_errors
#     from
#         payment_funnel_stages
#     group by
#         order_year
#     order by
#         order_year desc;
# """
# sql_query = jinja2.Template(raw_query).render(vars())

This query calculates overall payment funnel conversion metrics across all subscriptions.

It tells you:

* How many subscriptions completed payment
* How many started the payment process (but may not have finished)
* How many never started the payment process
* The total number of subscriptions
* The overall conversion rate (completed / total)
* The workflow conversion rate (completed / started)

In [ ]:
# import jinja2
# raw_query = """
#     with max_status_reached as(
#     select
#         psl.subscription_id,
#         max(psl.status_id) as max_status
#     from 
#         public.payment_status_log psl
#     group by    
#         1
#     )
#     ,
#     payment_funnel_stages as (
#     select
#         subs.subscription_id,
#         date_trunc('year' , order_date) as order_year,
#         current_payment_status,
#         max_status,
#         case when max_status = 5 then 1 else 0 end as completed_payment, 
#         case when max_status is not null then 1 else 0 end as started_payment,
#         case when max_status is null then 1 else 0 end as not_started_payment
#         
#     from
#         public.subscriptions subs
#     left join
#         max_status_reached m
#         on subs.subscription_id = m.subscription_id
#     )
#     select 
#         sum(completed_payment) as num_subs_completed_payment,
#         sum(started_payment) as num_subs_started_payment,
#         sum(not_started_payment) as num_subs_not_started_payment,
#         count(*)as total_subs,
#         num_subs_completed_payment / total_subs * 100 as conversion_rate,
#         num_subs_completed_payment / num_subs_started_payment * 100 as workflow_conversion_rate    
#     from 
#         payment_funnel_stages
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     -- Step 1: For each subscription, find the highest (max) payment status it ever reached.
#     with max_status_reached as (
#       select
#         psl.subscription_id,                    -- unique subscription
#         max(psl.status_id) as max_status        -- highest status_id that subscription reached
#       from public.payment_status_log psl
#       group by 1                                -- group by subscription_id
#     ),
#     
#     -- Step 2: Build a row per subscription with:
#     --   - the year of the order
#     --   - its current payment status
#     --   - the max status it reached
#     --   - flags for completed / started / not started payment
#     payment_funnel_stages as (
#       select
#         subs.subscription_id,                                -- unique subscription
#         date_trunc('year', order_date) as order_year,        -- order date truncated to the first day of its year
#         current_payment_status,                              -- current status of payment for this subscription
#         max_status,                                          -- highest status this subscription ever reached
#     
#         -- completed_payment: 1 if max_status = 5, else 0
#         -- (you’re treating status_id = 5 as "payment completed")
#         case when max_status = 5 then 1 else 0 end as completed_payment,
#     
#         -- started_payment: 1 if it has *any* payment status (not null), else 0
#         -- i.e., they entered the payment workflow at all
#         case when max_status is not null then 1 else 0 end as started_payment,
#     
#         -- not_started_payment: 1 if it never had any payment status (null), else 0
#         -- i.e., they never started the payment process
#         case when max_status is null then 1 else 0 end as not_started_payment
#     
#       from public.subscriptions subs
#       left join max_status_reached m
#         on subs.subscription_id = m.subscription_id          -- attach each subscription's max_status if it has one
#     ),
#     
#     -- Step 3: Aggregate counts by year into a "base" table.
#     -- This gives one row per order_year with total counts.
#     base as (
#       select
#         order_year,                                           -- the year bucket
#     
#         -- total number of subs that completed payment in that year
#         sum(completed_payment)       as num_subs_completed_payment,
#     
#         -- total number of subs that started payment in that year
#         sum(started_payment)         as num_subs_started_payment,
#     
#         -- total number of subs that never started payment in that year
#         sum(not_started_payment)     as num_subs_not_started_payment,
#     
#         -- total number of subscriptions in that year (all stages)
#         count(*)                     as total_subs
#     
#       from payment_funnel_stages
#       group by order_year
#     )
#     
#     -- Step 4: From the per-year counts in base, calculate conversion metrics.
#     select
#       order_year,                        -- the year
#       num_subs_completed_payment,        -- how many completed payment that year
#       num_subs_started_payment,          -- how many started payment that year
#       num_subs_not_started_payment,      -- how many did not start payment that year
#       total_subs,                        -- total subscriptions that year
#     
#       -- Overall conversion rate: completed / total, as a percentage.
#       -- ::float forces decimal math instead of integer math (so 3/5 = 0.6, not 0).
#       num_subs_completed_payment::float / total_subs::float * 100
#         as conversion_rate,
#     
#       -- Workflow conversion rate: completed / started, as a percentage.
#       -- nullif(num_subs_started_payment, 0) returns NULL if the denominator is 0,
#       -- which prevents a divide-by-zero error and yields NULL instead.
#       num_subs_completed_payment::float /
#         nullif(num_subs_started_payment, 0)::float * 100
#         as workflow_conversion_rate
#     
#     from base
#     order by order_year;                 -- sort results chronologically
# """
# sql_query = jinja2.Template(raw_query).render(vars())

This query provides a simple, high-level breakdown of subscriptions into two buckets:

Subscriptions that had at least one payment error
Subscriptions that had no payment errors
It’s essentially answering:

“Out of all subscriptions, how many ever experienced a payment error vs. how many never did?”

In [ ]:
# import jinja2
# raw_query = """
#     with error_subs as (
#       select distinct subscription_id
#       from public.payment_status_log
#       where status_id = 0
#     ),
#     agg as (
#       select
#         count(*) as total_subscriptions,
#         sum(case when err.subscription_id is not null then 1 else 0 end) as total_error_subs
#       from public.subscriptions subs
#       left join error_subs err
#         on subs.subscription_id = err.subscription_id
#     )
#     select 'Error'      as category,
#            total_error_subs as num_subs
#     from agg
#     union all
#     select 'Non-error'  as category,
#            total_subscriptions - total_error_subs as num_subs
#     from agg;
# """
# sql_query = jinja2.Template(raw_query).render(vars())